In [1]:
from pyspark.context import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql.functions import *
from pyspark.sql import *
from pyspark.sql.types import *
import os

In [2]:
spark = SparkSession.builder\
                    .master('local[2]')\
                    .appName('ch5')\
                    .getOrCreate()

In [3]:
df = spark.read.format("json")\
                .load(os.getcwd()+'/data/flight-data/json/2015-summary.json')

df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



In [9]:
df.schema

StructType(List(StructField(DEST_COUNTRY_NAME,StringType,true),StructField(ORIGIN_COUNTRY_NAME,StringType,true),StructField(count,LongType,true)))

In [27]:
mannualSchema = StructType([
    StructField("DEST_COUNTRY_NAME", StringType(), True),
    StructField("ORIGIN_COUNTRY_NAME", StringType(), True),
    StructField("count", StringType(), True, metadata={"hello":"world"})
])

df = spark.read.format("json")\
                .schema(mannualSchema)\
                .load(os.getcwd()+'/data/flight-data/json/2015-summary.json')

df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: string (nullable = true)



In [29]:
# 명시적 컬럼 참조
col('hi') # 미확인 상태

Column<'hi'>

In [24]:
# 전체 컬럼 정보 프린트
df.printSchema()

# 프로그래밍 방식의 컬럼 접근
df.columns

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: string (nullable = true)



['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count']

## 5.3 레코드와 로우

In [4]:
df.first()

Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15)

### 5.3.1 로우 생성하기

In [7]:
myRow = Row("Hello", None, 1, False)
print(myRow) # 스키마 존재 x
print(myRow[0])
print(myRow[1])

<Row('Hello', None, 1, False)>
Hello
None


### 5.4.1 DataFrame 생성하기

In [11]:
# case1
df = spark.read.format("json")\
                .load(os.getcwd()+'/data/flight-data/json/2015-summary.json')

df.createOrReplaceTempView("dfTable") # SQL

In [12]:
# case2
manualSchema = StructType([
    StructField("some", StringType(), True),
    StructField("col", StringType(), True),
    StructField("names", LongType(), True)
])

myRow = Row("hello", None, 1)
myDF = spark.createDataFrame([myRow], schema=manualSchema)
myDF.show()

+-----+----+-----+
| some| col|names|
+-----+----+-----+
|hello|null|    1|
+-----+----+-----+



### 5.4.2 select와 selectExpr

In [13]:
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



In [14]:
# 단일 컬럼 선택
df.select("DEST_COUNTRY_NAME").show(2, False)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|United States    |
|United States    |
+-----------------+
only showing top 2 rows



In [16]:
# 다중 컬럼 선택
df.select("DEST_COUNTRY_NAME", "ORIGIN_COUNTRY_NAME").show(2, False)

+-----------------+-------------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|
+-----------------+-------------------+
|United States    |Romania            |
|United States    |Croatia            |
+-----------------+-------------------+
only showing top 2 rows



In [18]:
# 컬럼을 참조하는 다양한 방법
df.select(
    expr("DEST_COUNTRY_NAME"),
    col("DEST_COUNTRY_NAME"),
    column("DEST_COUNTRY_NAME")
).show(2)

+-----------------+-----------------+-----------------+
|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|
+-----------------+-----------------+-----------------+
|    United States|    United States|    United States|
|    United States|    United States|    United States|
+-----------------+-----------------+-----------------+
only showing top 2 rows



In [20]:
# spark 2.0기준으로는 문자열과 column객체를 동시에 사용할 시에는 오류 반환
# 하지만, spark 3.0이상에서는 오류없이 정상 출력됨
# 그래도! 범용성을 위해서 아래와 같은 케이스는 피하도록 해야한다!
df.select(col("DEST_COUNTRY_NAME"), "DEST_COUNTRY_NAME").show(2)

+-----------------+-----------------+
|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|
+-----------------+-----------------+
|    United States|    United States|
|    United States|    United States|
+-----------------+-----------------+
only showing top 2 rows



In [24]:
# 별칭1
df.select(expr("DEST_COUNTRY_NAME AS dest")).show(2)

# 별칭2
df.select(expr("DEST_COUNTRY_NAME AS dest").alias("DEST_COUNTRY_NAME")).show(2)

+-------------+
|         dest|
+-------------+
|United States|
|United States|
+-------------+
only showing top 2 rows

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows



In [26]:
# selectExpr1
df.select(expr("DEST_COUNTRY_NAME AS dest"), expr("DEST_COUNTRY_NAME")).show(2)
df.selectExpr("DEST_COUNTRY_NAME AS dest", "DEST_COUNTRY_NAME").show(2)

+-------------+-----------------+
|         dest|DEST_COUNTRY_NAME|
+-------------+-----------------+
|United States|    United States|
|United States|    United States|
+-------------+-----------------+
only showing top 2 rows

+-------------+-----------------+
|         dest|DEST_COUNTRY_NAME|
+-------------+-----------------+
|United States|    United States|
|United States|    United States|
+-------------+-----------------+
only showing top 2 rows



In [28]:
# selectExpr2
df.selectExpr(
    "*",
    "(DEST_COUNTRY_NAME=ORIGIN_COUNTRY_NAME) as withinCountry"
).show(2)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



In [32]:
# selectExpr3 : 집계 함수 지정 가능
df.selectExpr("avg(count)", "count(distinct(DEST_COUNTRY_NAME))").show(2)

# 물론 expr함수를 사용해서도 표현가능하지만, selectExpr로 통일하는게 깔끔
df.select(expr("avg(count)"), expr("count(distinct(DEST_COUNTRY_NAME))")).show(2)

# col함수로는 위와 같이 표현 불가능

+-----------+---------------------------------+
| avg(count)|count(DISTINCT DEST_COUNTRY_NAME)|
+-----------+---------------------------------+
|1770.765625|                              132|
+-----------+---------------------------------+

+-----------+---------------------------------+
| avg(count)|count(DISTINCT DEST_COUNTRY_NAME)|
+-----------+---------------------------------+
|1770.765625|                              132|
+-----------+---------------------------------+



### 5.4.3 스파크 데이터타입으로 변환하기

In [34]:
df.select(expr("*"), lit(1).alias("One")).show(2)

+-----------------+-------------------+-----+---+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|One|
+-----------------+-------------------+-----+---+
|    United States|            Romania|   15|  1|
|    United States|            Croatia|    1|  1|
+-----------------+-------------------+-----+---+
only showing top 2 rows



### 5.4.4 컬럼 추가하기

In [37]:
# before
df.show(2)
# after
df.withColumn("numberOne", lit(1)).show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
+-----------------+-------------------+-----+
only showing top 2 rows

+-----------------+-------------------+-----+---------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|numberOne|
+-----------------+-------------------+-----+---------+
|    United States|            Romania|   15|        1|
|    United States|            Croatia|    1|        1|
+-----------------+-------------------+-----+---------+
only showing top 2 rows



In [38]:
df.withColumn("withinCountry", expr("ORIGIN_COUNTRY_NAME==DEST_COUNTRY_NAME")).show(2)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



In [39]:
# 동일 record의 내용을 가진 컬럼을 다른 이름으로 추가
df.withColumn('Destination', expr("DEST_COUNTRY_NAME")).columns

['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count', 'Destination']

### 5.4.5 컬럼명 변경하기

In [40]:
# before
df.show(2)
# after
df.withColumnRenamed("DEST_COUNTRY_NAME", "dest").show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
+-----------------+-------------------+-----+
only showing top 2 rows

+-------------+-------------------+-----+
|         dest|ORIGIN_COUNTRY_NAME|count|
+-------------+-------------------+-----+
|United States|            Romania|   15|
|United States|            Croatia|    1|
+-------------+-------------------+-----+
only showing top 2 rows



### 5.4.6 예약 문자와 키워드

In [42]:
dfWithLongColName = df.withColumn("This Long Column-Name", expr("ORIGIN_COUNTRY_NAME")) # 문자열로 사용하는 케이스에서는 예약문자를 이스케이핑시킬 필요 없음
dfWithLongColName.selectExpr("`This Long Column-Name`", "`This Long Column-Name` as `new col`").show(2) # 표현식으로 사용하는 케이스에서는 예약문자를 이스케이핑 처리해야함

+---------------------+-------+
|This Long Column-Name|new col|
+---------------------+-------+
|              Romania|Romania|
|              Croatia|Croatia|
+---------------------+-------+
only showing top 2 rows



### 5.4.7 대소문자 구분

In [ ]:
# 스파크는 대소문자를 구분하지 않는다. 하지만 다음의 설정으로 구분하게 할 수 있다.
spark.conf.set('spark.sql.caseSensitive', True)

### 5.4.8 컬럼제거하기

In [46]:
# 단일 제거
df.drop("ORIGIN_COUNTRY_NAME").columns

['DEST_COUNTRY_NAME', 'count']

In [48]:
# 다중 제거
df.drop("ORIGIN_COUNTRY_NAME", "DEST_COUNTRY_NAME").columns

['count']

### 5.4.9 컬럼의 데이터 타입 변경하기

In [51]:
# before
df.printSchema()
# after
df.withColumn("count2", col("count").cast("string")).printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)
 |-- count2: string (nullable = true)



### 5.4.10 로우 필터링하기

In [53]:
df.filter(col('count')<2).show(2)
df.where(col('count')<2).show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



In [54]:
# AND필터링
df.where(col('count')<2)\
    .where(col("ORIGIN_COUNTRY_NAME") != "Croatia")\
        .show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|          Singapore|    1|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



### 5.4.11 고유한 로우 얻기

In [55]:
df.select("ORIGIN_COUNTRY_NAME", "DEST_COUNTRY_NAME").distinct().count()

256

In [56]:
df.select("ORIGIN_COUNTRY_NAME").distinct().count()

125

### 5.4.12 무작위 샘플 만들기

In [5]:
seed = 5
withReplacement = False
fraction = 0.5
print('original  :', df.count())
print('sample : ', df.sample(withReplacement, fraction, seed).count())

original  : 256
sample :  138


### 5.4.13 임의 분할하기

In [8]:
dataframes = df.randomSplit([0.25, 0.75], seed)
print('25% : ', dataframes[0].count())
print('75% : ', dataframes[1].count())

25% :  71
75% :  185


### 5.4.14 로우 합치기와 추가하기

In [14]:
schema = StructType([StructField("c1", StringType(), False),
                    StructField("c2", StringType(), False),
                    StructField("c3", StringType(), False)])

newdf1 = spark.createDataFrame([Row("col1_1", "col2_1", "col3_1")], schema)
newdf2 = spark.createDataFrame([Row("col1_2", "col2_2", "col3_2")], schema)

newdf1.show()
newdf2.show()
newdf1.union(newdf2).show()

+------+------+------+
|    c1|    c2|    c3|
+------+------+------+
|col1_1|col2_1|col3_1|
+------+------+------+

+------+------+------+
|    c1|    c2|    c3|
+------+------+------+
|col1_2|col2_2|col3_2|
+------+------+------+

+------+------+------+
|    c1|    c2|    c3|
+------+------+------+
|col1_1|col2_1|col3_1|
|col1_2|col2_2|col3_2|
+------+------+------+

